In [1]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

In [2]:
driver_path = 'C:/chromedriver_win32/chromedriver.exe' 

In [3]:
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_tex

In [4]:
def m_i(query,driver_path,page):
    domain = '매일일보'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
    
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    data_list = []
    head_list = []
    
    while current_page < last_page:
        print(current_page)
        driver = webdriver.Chrome(executable_path =driver_path, options=options)
        url_domain ='https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2385&nso=so%3Ar%2Cp%3Aall%2Ca%3Aall&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        print(url)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
    
    # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
    
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
    
        except: 
            print('기사가 없습니다.')
            driver.close()
            return ()
        driver.close()
    
    
        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
            
            try:
                driver.get(url)
                time.sleep(1)
            except:
                print('link error')
                continue
        # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select('#user-container > div.custom-pc.float-center.max-width-1130 > header > div > div')[0].text)
                tags = soup.find(id='article-view-content-div').find_all('p')
                data = [x.text for x in tags]
                data_list.append(' '.join(data))
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page += 10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

mi = m_i('풀무원 "바른먹거리"',driver_path,3)

1
https://search.naver.com/search.naver?where=news&query=풀무원 "바른먹거리"&%2Ca%3A&start=1&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2385&nso=so%3Ar%2Cp%3Aall%2Ca%3Aall&is_sug_officeid=0
11
https://search.naver.com/search.naver?where=news&query=풀무원 "바른먹거리"&%2Ca%3A&start=11&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2385&nso=so%3Ar%2Cp%3Aall%2Ca%3Aall&is_sug_officeid=0


In [5]:
mi.to_csv('매일일보.csv',encoding='utf-8-sig',index=None)
test = pd.read_csv('매일일보.csv', encoding= 'utf-8-sig')

In [6]:
test

,언론사,제목,내용
0,매일일보,"윤희선 풀무원푸드머스 대표, ‘어린이 교통안전 릴레이 챌린지’ 동참",[매일일보 나기호 기자] 윤희선 풀무원푸드머스 대표가 어린이 보호구역 내 교통안전 ...
1,매일일보,"풀무원푸드머스, 경기도국공립어린이집연합회와 식습관 MOU 체결",[매일일보 황양택 기자] 풀무원 계열 식자재 유통기업 풀무원푸드머스는 경기도국공립어...
2,매일일보,"풀무원푸드머스, 전국시군구육아종합지원센터 협의회와 MOU 체결",[매일일보 황양택 기자] 풀무원 계열 식자재 유통기업 풀무원푸드머스(는 전국시군구육...
3,매일일보,"풀무원, 국산콩두부 ‘글로벌 탄소발자국 인증’ 획득… 美 수출 두부도 추진",[매일일보 나기호 기자] 풀무원은 국산콩두부 10종이 영국 친환경 인증기관 ‘카본 ...
4,매일일보,"풀무원푸드머스, ‘킨더슐레보육경영연구소’와 바른먹거리 MOU 체결",[매일일보 전지현 기자] 풀무원 계열 식자재 유통기업 ㈜풀무원푸드머스는 국내 대표 ...
5,매일일보,"풀무원, 14년 연속 ‘한국에서 가장 존경받는 기업’ 선정",[매일일보 임유정 기자] 풀무원이 14년 연속 ‘한국에서 가장 존경받는 기업’에 선...
6,매일일보,"풀무원, 냉동HMR 간식 신제품 ‘모짜렐라·앙크림 치즈볼’ 2종 출시",[매일일보 임유정 기자] 풀무원이 간식 신제품 ‘냉동 치즈볼’을 선보인다. 풀무원식...
7,매일일보,"풀무원 시니어 바른먹거리 교육, 대사증후군 32.6% 개선 효과",[매일일보 임유정 기자] 풀무원은 지난해 농촌 지역 시니어 식생활 개선 사업 ‘시니...
8,매일일보,"풀무원, ‘2019년 DJSI평가’ 글로벌 식품 기업 중 2년 연속 10위권",[매일일보 임유정 기자] 풀무원이 다우존스가 평가하는 지속경영지수(DJSI) 평가에...
9,매일일보,"풀무원 올가홀푸드, 친환경 농업 공로 '농림축산식품부 장관 표창' 수상",[매일일보 황양택 기자] 풀무원은 로하스 프레시 마켓(LOHAS Fresh Mark...
